# Analyse

In [1]:
import pandas as pd
import time, requests, re
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns 
sns.set(rc={'figure.figsize':(11.7,8.27)})
plt.style.use('ggplot')
# sns.set_style("whitegrid", {'axes.grid' : True})

In [27]:
#The data with the first differences is loaded
path = "https://github.com/Naeim-S/GRP33/blob/master/EXAM/Multi_JobIndexed.xlsx?raw=true"
jobindexed = pd.read_excel(path)
#Multi indexing will be done after merging:
# jobindexed = jobindexed.set_index(['year', 'area', 'sektor', 'jobtype']).sort_index()
jobindexed.tail(10)

,year,area,sektor,jobtype,n_jobtype_area,n_sektor_area,n_sektor,n_area,n_year,20-64years_area,20-64years_year,n_jobtype_area_pc,n_sektor_area_pc,n_area_pc,n_year_pc,FD_jobtype_area,FD_sektor_area,FD_sektor,FD_area,FD_year
4940,2017,region-sjaelland,social,pleje,2431,6330,45693,25973,266462,462645,3341882,0.005255,0.013682,0.05614,0.079734,292.0,-10.0,1349.0,-368.0,-5600.0
4941,2017,region-sjaelland,social,psykologi,197,6330,45693,25973,266462,462645,3341882,0.000426,0.013682,0.05614,0.079734,21.0,-10.0,1349.0,-368.0,-5600.0
4942,2017,region-sjaelland,social,socialraadgivning,487,6330,45693,25973,266462,462645,3341882,0.001053,0.013682,0.05614,0.079734,52.0,-10.0,1349.0,-368.0,-5600.0
4943,2017,region-sjaelland,social,tandlaege,232,6330,45693,25973,266462,462645,3341882,0.000501,0.013682,0.05614,0.079734,21.0,-10.0,1349.0,-368.0,-5600.0
4944,2017,region-sjaelland,social,teknisksundhed,77,6330,45693,25973,266462,462645,3341882,0.000166,0.013682,0.05614,0.079734,12.0,-10.0,1349.0,-368.0,-5600.0
4945,2017,region-sjaelland,social,terapi,360,6330,45693,25973,266462,462645,3341882,0.000778,0.013682,0.05614,0.079734,-3.0,-10.0,1349.0,-368.0,-5600.0
4946,2017,region-sjaelland,undervisning,bibliotek,40,4499,43176,25973,266462,462645,3341882,0.000087,0.009725,0.05614,0.079734,17.0,231.0,985.0,-368.0,-5600.0
4947,2017,region-sjaelland,undervisning,forskning,594,4499,43176,25973,266462,462645,3341882,0.001284,0.009725,0.05614,0.079734,131.0,231.0,985.0,-368.0,-5600.0
4948,2017,region-sjaelland,undervisning,laerer,1475,4499,43176,25973,266462,462645,3341882,0.003188,0.009725,0.05614,0.079734,-156.0,231.0,985.0,-368.0,-5600.0
4949,2017,region-sjaelland,undervisning,paedagog,2197,4499,43176,25973,266462,462645,3341882,0.004749,0.009725,0.05614,0.079734,236.0,231.0,985.0,-368.0,-5600.0


In [28]:
#Load the BNP data from DST:
import requests

def get_data(table_id,variables):
    base = 'https://api.statbank.dk/v1/data/{id}/JSONSTAT?lang=en'.format(id = table_id)
    
    for var in variables:
        base += '&{v}'.format(v = var) 

    response=requests.get(base)
    data_json=response.json()
    return data_json

indexlabels_bnp=[2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017]
data_bnp=get_data('NAN1',['Tid=2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017','TRANSAKT=B1GQK','PRISENHED=LAN_C']) # Nordjylland
BNP=pd.DataFrame(data_bnp['dataset']['value'],index=indexlabels_bnp,columns=['BNP, 1000 kr. (2010)'])
# BNP=BNP.reindex(labels = ['year'], index=BNP.index[::-1]).sort_index()
# BNP=BNP.transpose()
BNP.index.names = ['year']
BNP["FD_BNP"] = BNP.diff(1)
BNP

,"BNP, 1000 kr. (2010)",FD_BNP
year,,
2007,344.115676,NaN
2008,340.338850,-3.776826
2009,321.889946,-18.448904
2010,326.448008,4.558062
2011,329.454843,3.006835
2012,328.986910,-0.467933
2013,330.726886,1.739976
2014,334.291100,3.564214
2015,337.324447,3.033347


In [34]:
#Merge the JobIndex data with the BNP:
merged = pd.merge(jobindexed, BNP, on="year", how="left", validate="m:1")
merged.to_csv("merged.csv") #Saved as a back up, just in case.
#Create the multi index:
merged = merged.set_index(['year', 'area', 'sektor', 'jobtype']).sort_index()
merged.tail(10) #Looks beautiful

n_jobtype_area  \
year area             sektor       jobtype                             
2017 region-sjaelland social       pleje                        2431   
                                   psykologi                     197   
                                   socialraadgivning             487   
                                   tandlaege                     232   
                                   teknisksundhed                 77   
                                   terapi                        360   
                      undervisning bibliotek                      40   
                                   forskning                     594   
                                   laerer                       1475   
                                   paedagog                     2197   

                                                      n_sektor_area  n_sektor  \
year area             sektor       jobtype                                      
2017 region-sjaelland social       pleje                       6330     45693   
                                   psykologi                   6330     45693   
                                   socialraadgivning           6330     45693   
                                   tandlaege                   6330     45693   
                                   teknisksundhed              6330     45693   
                                   terapi                      6330     45693   
                      undervisning bibliotek                   4499     43176   
                                   forskning                   4499     43176   
                                   laerer                      4499     43176   
                                   paedagog                    4499     43176   

                                                      n_area  n_year  \
year area             sektor       jobtype                             
2017 region-sjaelland social       pleje               25973  266462   
                                   psykologi           25973  266462   
                                   socialraadgivning   25973  266462   
                                   tandlaege           25973  266462   
                                   teknisksundhed      25973  266462   
                                   terapi              25973  266462   
                      undervisning bibliotek           25973  266462   
                                   forskning           25973  266462   
                                   laerer              25973  266462   
                                   paedagog            25973  266462   

                                                      20-64years_area  \
year area             sektor       jobtype                              
2017 region-sjaelland social       pleje                       462645   
                                   psykologi                   462645   
                                   socialraadgivning           462645   
                                   tandlaege                   462645   
                                   teknisksundhed              462645   
                                   terapi                      462645   
                      undervisning bibliotek                   462645   
                                   forskning                   462645   
                                   laerer                      462645   
                                   paedagog                    462645   

                                                      20-64years_year  \
year area             sektor       jobtype                              
2017 region-sjaelland social       pleje                      3341882   
                                   psykologi                  3341882   
                                   socialraadgivning          3341882   
                                   tandlaege                  3341882   
                                   t

In [41]:
#Creating the ratio variables:
merged["ratio_jobtype_area"] = merged["FD_jobtype_area"] / merged["FD_BNP"]
merged["ratio_sektor_area"] = merged["FD_sektor_area"] / merged["FD_BNP"]
merged["ratio_sektor"] = merged["FD_sektor"] / merged["FD_BNP"]
merged["ratio_area"] = merged["FD_area"] / merged["FD_BNP"]
merged["ratio_year"] = merged["FD_year"] / merged["FD_BNP"]
merged.tail(10) #What a lovely sight

n_jobtype_area  \
year area             sektor       jobtype                             
2017 region-sjaelland social       pleje                        2431   
                                   psykologi                     197   
                                   socialraadgivning             487   
                                   tandlaege                     232   
                                   teknisksundhed                 77   
                                   terapi                        360   
                      undervisning bibliotek                      40   
                                   forskning                     594   
                                   laerer                       1475   
                                   paedagog                     2197   

                                                      n_sektor_area  n_sektor  \
year area             sektor       jobtype                                      
2017 region-sjaelland social       pleje                       6330     45693   
                                   psykologi                   6330     45693   
                                   socialraadgivning           6330     45693   
                                   tandlaege                   6330     45693   
                                   teknisksundhed              6330     45693   
                                   terapi                      6330     45693   
                      undervisning bibliotek                   4499     43176   
                                   forskning                   4499     43176   
                                   laerer                      4499     43176   
                                   paedagog                    4499     43176   

                                                      n_area  n_year  \
year area             sektor       jobtype                             
2017 region-sjaelland social       pleje               25973  266462   
                                   psykologi           25973  266462   
                                   socialraadgivning   25973  266462   
                                   tandlaege           25973  266462   
                                   teknisksundhed      25973  266462   
                                   terapi              25973  266462   
                      undervisning bibliotek           25973  266462   
                                   forskning           25973  266462   
                                   laerer              25973  266462   
                                   paedagog            25973  266462   

                                                      20-64years_area  \
year area             sektor       jobtype                              
2017 region-sjaelland social       pleje                       462645   
                                   psykologi                   462645   
                                   socialraadgivning           462645   
                                   tandlaege                   462645   
                                   teknisksundhed              462645   
                                   terapi                      462645   
                      undervisning bibliotek                   462645   
                                   forskning                   462645   
                                   laerer                      462645   
                                   paedagog                    462645   

                                                      20-64years_year  \
year area             sektor       jobtype                              
2017 region-sjaelland social       pleje                      3341882   
                                   psykologi                  3341882   
                                   socialraadgivning          3341882   
                                   tandlaege                  3341882   
                                   t